# PDI - Rebalancing data for August 2022

## Setting

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
from defillama import DefiLlama
llama = DefiLlama()

In [4]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    
    
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    1. The token must not have locking, minting or other patterns that would significantly disadvantage passive holders.


1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least $5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The token must have shown consistent DeFi liquidity on Ethereum.


1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [6]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [7]:
protocols = llama.get_all_protocols()

In [8]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': pr['address'].lower()
        }

#### CoinGecko

##### Tokens addresses

In [9]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [10]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [11]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [12]:
tokens['BNT']

{'symbol': 'BNT', 'address': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c'}

In [13]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        print(token)
        exclude_symbols.append(token['symbol'])

{'symbol': 'MULTI', 'address': '0x9fb9a33956351cf4fa040f65a13b835a3c8764e3'}
{'symbol': 'IB', 'address': 'fantom:0x00a35fd824c717879bf370e70ac6868b95870dfb'}
{'symbol': 'COW', 'address': 'heco:0x80861a817106665bca173db6ac2ab628a738c737'}
{'symbol': 'PINKSALE', 'address': 'bsc:0x602ba546a7b06e0fc7f58fd27eb6996ecc824689'}
{'symbol': 'ABR', 'address': 'celo:0x6e512bfc33be36f2666754e996ff103ad1680cc9'}
{'symbol': 'FLAG', 'address': '0x57dd84042ec9507963016596a34edad42f7e4cce '}
{'symbol': 'HEC', 'address': 'fantom:0x5c4fdfc5233f935f20d2adba572f770c2e377ab0'}
{'symbol': 'RSR', 'address': '0x8762db106b2c2a0bccb3a80d1ed41273552616e8'}
{'symbol': 'SGT', 'address': '0x84810bcf08744d5862b8181f12d17bfd57d3b078'}
{'symbol': 'FHM', 'address': '0xfa1fbb8ef55a4855e5688c0ee13ac3f202486286'}
{'symbol': 'GEAR', 'address': '0xba3335588d9403515223f109edc4eb7269a9ab5d'}
{'symbol': 'SWPR', 'address': 'arbitrum:0xde903e2712288a1da82942dddf2c20529565ac30'}
{'symbol': 'SOLV', 'address': '0x256f2d67e52fe834726d

In [14]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

In [15]:
# KEEP has this address in defillama, but etherscan says this address belongs to tBTC now
# for token in top_defi_tokens.values():
#     if token['address'] == "0x8daebade922df735c38c80c7ebd708af50815faa":
#         print(token)
# del top_defi_tokens['KEEP']

##### Market data

In [16]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [17]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [18]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [19]:
tokens_market_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,2.295100e+04,438503683790,1,4.819914e+11,5.396540e+10,2.300100e+04,...,2.100000e+07,2.100000e+07,6.904500e+04,-66.75785,2021-11-10T14:24:11.849Z,6.781000e+01,3.374797e+04,2013-07-06T00:00:00.000Z,None,2022-07-27T19:48:35.898Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1.600670e+03,192050123810,2,NaN,2.199940e+10,1.603320e+03,...,1.198510e+08,NaN,4.878260e+03,-67.15198,2021-11-10T14:24:19.604Z,4.329790e-01,3.699901e+05,2015-10-20T00:00:00.000Z,"{'times': 92.22534026457839, 'currency': 'btc'...",2022-07-27T19:49:15.968Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.006000e+00,66227147394,3,NaN,6.606134e+10,1.007000e+00,...,6.587632e+10,NaN,1.320000e+00,-24.01715,2018-07-24T00:00:00.000Z,5.725210e-01,7.559628e+01,2015-03-02T00:00:00.000Z,None,2022-07-27T19:45:56.815Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.006000e+00,55492377790,4,NaN,7.792165e+09,1.008000e+00,...,5.511566e+10,NaN,1.170000e+00,-14.15840,2019-05-08T00:40:28.300Z,8.918480e-01,1.287458e+01,2021-05-19T13:14:05.611Z,None,2022-07-27T19:49:02.510Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,2.694500e+02,44011365587,5,4.450728e+10,1.292291e+09,2.697500e+02,...,1.632770e+08,1.651168e+08,6.863100e+02,-60.72450,2021-05-10T07:24:17.097Z,3.981770e-02,6.768612e+05,2017-10-19T00:00:00.000Z,None,2022-07-27T19:48:52.814Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EMT,emanate,emt,Emanate,https://assets.coingecko.com/coins/images/4152...,7.674730e-03,1146045,1891,NaN,1.680900e+02,7.735280e-03,...,2.080000e+08,NaN,1.961590e-01,-96.08750,2020-03-09T02:34:55.973Z,9.383000e-09,8.179410e+07,2020-08-09T14:14:06.411Z,"{'times': -0.9360438807359804, 'currency': 'us...",2022-07-27T06:33:55.670Z
VHC,vault-hill-city,vhc,Vault Hill City,https://assets.coingecko.com/coins/images/2312...,1.205801e-02,1140521,1894,4.094331e+06,7.612140e+05,1.221439e-02,...,3.400000e+08,3.400000e+08,8.189800e-02,-85.29898,2022-03-29T07:30:14.828Z,1.125334e-02,6.989140e+00,2022-05-14T18:29:16.226Z,None,2022-07-27T19:49:54.834Z
SWD,sw-dao,swd,SW DAO,https://assets.coingecko.com/coins/images/1933...,2.830000e+00,1137082,1896,2.791787e+06,2.776000e+01,2.830000e+00,...,9.853580e+05,9.853580e+05,2.272000e+01,-87.54996,2022-02-21T12:22:39.636Z,1.550000e+00,8.268802e+01,2022-04-18T16:59:25.630Z,None,2022-07-27T19:47:35.789Z


In [20]:
tokens_market_data.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

### Token's supply check 

In [21]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]

### Liquidity check

In [22]:
api_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'univ3': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3",
    'sushi': "https://api.thegraph.com/subgraphs/name/sushiswap/exchange",
    'bancorv2': "https://api-v2.bancor.network/tokens",
    'bancorv3': "https://api-v3.bancor.network/pools",
    'balancer': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer",
    'balancerv2': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2" 
} 

In [23]:
def univ2_get_liquidities(tokens_ids):
    query = """
        query tokens ($tokens_ids: [ID!]!) {
            tokens (first:1000, where: {id_in: $tokens_ids}){
                id
                symbol
                totalLiquidity
            }
        }
    """
    url = api_urls['univ2']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    tokens = response.json()['data']['tokens']
    
    liqs = {}
    for token in tokens:
        symb = normalize_symb(token['symbol'])
        liqs[symb] = float(token['totalLiquidity'])
        
    return liqs
    

In [24]:
def univ3_get_liquidities(tokens_ids):
    query = """
        query tokens ($tokens_ids: [ID!]!) {
            tokens (first:1000, where: {id_in: $tokens_ids}){
                id
                symbol
                totalValueLockedUSD
            }
        }
    """
    url = api_urls['univ3']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    tokens = response.json()['data']['tokens']
    
    liqs = {}
    for token in tokens:
        symb = normalize_symb(token['symbol'])
        liqs[symb] = float(token['totalValueLockedUSD'])
        
    return liqs
    

In [25]:
def sushi_get_liquidities(tokens_ids):
#     query = """
#         query tokens ($tokens_ids: [id!]!) {
#             tokens (first:1000, where: {id_in: $tokens_ids}){
#                 id
#                 symbol
#                 liquidity
#             }
#         }
#     """
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, where: {token0_in: $tokens_ids, token1_in: $tokens_ids}){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserve0
                reserve1
            }
        }
    """
    url = api_urls['sushi']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        symb0 = normalize_symb(pair['token0']['symbol'])
        if symb0 in tokens_market_data.index and pair['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + float(pair['reserve0']) * tokens_market_data.loc[symb0]['current_price']
        
        symb1 = normalize_symb(pair['token1']['symbol'])
        if symb1 in tokens_market_data.index and pair['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + float(pair['reserve1']) * tokens_market_data.loc[symb1]['current_price']
        
    return liqs
    

In [26]:
def bancorv2_get_liquidities(tokens_ids):
    url = api_urls['bancorv2'] 
    response = requests.get(url)
    tokens = response.json()['data']
    
    liqs = {}
    for token in tokens:
        if token['dlt_id'].lower() in tokens_ids:
            symb = normalize_symb(token['symbol'])
            liqs[symb] = float(token['liquidity']['usd'])
            
    return liqs
            

In [27]:
def bancorv3_get_liquidities(tokens_ids):
    url = api_urls['bancorv3'] 
    response = requests.get(url)
    pools = response.json()['data']
    
    liqs = {}
    for pool in pools:
        if pool['poolDltId'].lower() in tokens_ids:
            symb = normalize_symb(pool['name'])
            liqs[symb] = liqs.get(symb, 0) + float(pool['liquidity']['usd'])
            
    return liqs
            

In [28]:
def balancer_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  liquidity_gt: 10000,
                  
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
          }
        }
    """
    url = api_urls['balancer']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
            
            liqs[symb] = liqs.get(symb, 0) + float(token['balance']) * tokens_market_data.loc[symb]['current_price']
        
    return liqs
    

In [29]:
def balancerv2_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  totalLiquidity_gt: 10000,
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
          }
        }
    """
    url = api_urls['balancerv2']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
            
            liqs[symb] = liqs.get(symb, 0) + float(token['balance']) * tokens_market_data.loc[symb]['current_price']
        
    return liqs
    

In [30]:
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [31]:
tokens_ids.append(tokens['WETH']['address'])
tokens_ids.append(tokens['USDC']['address'])

In [32]:
query_functions = {
    'univ2': univ2_get_liquidities,
    'univ3': univ3_get_liquidities, 
    'sushi': sushi_get_liquidities, 
    'bancorv2': bancorv2_get_liquidities,
    'bancorv3': bancorv3_get_liquidities,
    'balancer': balancer_get_liquidities, 
    'balancerv2': balancerv2_get_liquidities, 
}

In [33]:
liqs_dict = {}
for dex, func in query_functions.items():
    liqs_dict[dex] = func(tokens_ids)

In [34]:
liquidities = pd.DataFrame.from_dict(liqs_dict)

In [35]:
liquidities['sum'] = liquidities.sum(axis=1)
liquidities.sort_values('sum', ascending=False)

,univ2,univ3,sushi,bancorv2,bancorv3,balancer,balancerv2,sum
USDC,2.060827e+08,1.542676e+09,1.883447e+07,834087.615312,1.449865e+06,8.821770e+06,2.284192e+07,1.801540e+09
BAO,1.727429e+09,0.000000e+00,3.420281e+05,NaN,NaN,NaN,NaN,1.727771e+09
WETH,3.308447e+05,7.951631e+08,NaN,NaN,NaN,NaN,NaN,7.954939e+08
LFT,6.564898e+08,0.000000e+00,NaN,NaN,NaN,NaN,NaN,6.564898e+08
RISE,6.091818e+08,NaN,NaN,NaN,NaN,NaN,NaN,6.091818e+08
...,...,...,...,...,...,...,...,...
WEB3,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00
FOREX,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00
VIRTUE,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00
BUMP,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00


In [36]:
liq_check = (liquidities['sum'] > 5e6).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [37]:
lq_passed = sp_passed[
    sp_passed.index.isin(liq_check_passed.index)
]

In [38]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.006000,55492377790,4,NaN,7.792165e+09,1.008000,...,5.511566e+10,NaN,1.170000,-14.15840,2019-05-08T00:40:28.300Z,8.918480e-01,1.287458e+01,2021-05-19T13:14:05.611Z,None,2022-07-27T19:49:02.510Z
WBTC,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,22990.000000,5421949596,18,5.421950e+09,3.835261e+08,23014.000000,...,2.364103e+05,2.364103e+05,70643.000000,-67.49441,2021-11-10T14:40:19.650Z,3.139170e+03,6.315000e+02,2019-04-02T00:00:00.000Z,None,2022-07-27T19:49:25.115Z
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.900000,3625909763,24,7.943028e+09,3.009469e+08,7.940000,...,1.000000e+09,1.000000e+09,44.920000,-82.33193,2021-05-03T05:25:04.822Z,1.030000e+00,6.703728e+02,2020-09-17T01:20:38.214Z,None,2022-07-27T19:48:37.576Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,93.530000,1308003999,47,1.498136e+09,2.348625e+08,93.680000,...,1.600000e+07,1.600000e+07,661.690000,-85.84669,2021-05-18T21:19:59.514Z,2.602000e+01,2.598761e+02,2020-11-05T09:20:11.928Z,None,2022-07-27T19:48:52.550Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.850000,955032606,56,1.853511e+09,1.036694e+08,1.860000,...,1.000000e+09,1.000000e+09,7.300000,-74.59839,2021-08-20T08:35:20.158Z,4.061500e-01,3.567396e+02,2022-06-18T20:55:12.035Z,None,2022-07-27T19:48:59.813Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,996.630000,898453333,57,1.002389e+09,9.159777e+07,997.430000,...,9.776310e+05,1.005577e+06,6292.310000,-84.15798,2021-05-03T21:54:29.333Z,1.683600e+02,4.920907e+02,2020-03-16T20:52:36.527Z,None,2022-07-27T19:49:00.310Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008655,659995805,78,8.614101e+08,5.768600e+06,0.008687,...,9.944413e+10,9.922516e+10,0.120813,-92.81796,2021-06-16T16:03:41.147Z,2.505010e-03,2.463779e+02,2020-11-04T13:18:12.191Z,None,2022-07-27T19:49:03.308Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,6.220000,449206940,97,6.213085e+08,5.643365e+06,6.220000,...,9.982298e+07,9.982298e+07,42.800000,-85.46293,2022-01-12T15:22:27.465Z,1.500000e+00,3.139170e+02,2021-06-25T16:50:51.447Z,None,2022-07-27T19:49:12.665Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.683627,385296028,111,1.026095e+09,3.664180e+07,0.683945,...,1.500000e+09,1.500000e+09,8.650000,-92.09676,2021-10-27T08:24:54.808Z,5.192810e-01,3.167418e+01,2022-06-18T20:10:15.819Z,None,2022-07-27T19:49:52.412Z


### Non quantative checks

In [39]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    },
    {
        'symbol': 'USDC',
        'reason': 'stable'
    },
    {
        'symbol': 'FEI',
        'reason': 'stable'
    },
    {
        'symbol': 'BNT',
        'reason': 'Token has a circular feedback loop with current ILP implementation'
    }
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [40]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [41]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.900000,3625909763,24,7.943028e+09,300946916.0,7.940000,...,1.000000e+09,1.000000e+09,44.920000,-82.33193,2021-05-03T05:25:04.822Z,1.030000e+00,6.703728e+02,2020-09-17T01:20:38.214Z,None,2022-07-27T19:48:37.576Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,93.530000,1308003999,47,1.498136e+09,234862484.0,93.680000,...,1.600000e+07,1.600000e+07,661.690000,-85.84669,2021-05-18T21:19:59.514Z,2.602000e+01,2.598761e+02,2020-11-05T09:20:11.928Z,None,2022-07-27T19:48:52.550Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.850000,955032606,56,1.853511e+09,103669374.0,1.860000,...,1.000000e+09,1.000000e+09,7.300000,-74.59839,2021-08-20T08:35:20.158Z,4.061500e-01,3.567396e+02,2022-06-18T20:55:12.035Z,None,2022-07-27T19:48:59.813Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,996.630000,898453333,57,1.002389e+09,91597770.0,997.430000,...,9.776310e+05,1.005577e+06,6292.310000,-84.15798,2021-05-03T21:54:29.333Z,1.683600e+02,4.920907e+02,2020-03-16T20:52:36.527Z,None,2022-07-27T19:49:00.310Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008655,659995805,78,8.614101e+08,5768600.0,0.008687,...,9.944413e+10,9.922516e+10,0.120813,-92.81796,2021-06-16T16:03:41.147Z,2.505010e-03,2.463779e+02,2020-11-04T13:18:12.191Z,None,2022-07-27T19:49:03.308Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.683627,385296028,111,1.026095e+09,36641795.0,0.683945,...,1.500000e+09,1.500000e+09,8.650000,-92.09676,2021-10-27T08:24:54.808Z,5.192810e-01,3.167418e+01,2022-06-18T20:10:15.819Z,None,2022-07-27T19:49:52.412Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,53.010000,361079107,116,5.266549e+08,107848420.0,52.970000,...,1.000000e+07,1.000000e+07,910.540000,-94.21283,2021-05-12T02:29:08.794Z,2.641000e+01,9.952380e+01,2022-06-18T20:54:18.261Z,None,2022-07-27T19:49:52.508Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.350000,260565590,137,3.378891e+08,78415560.0,1.350000,...,2.435460e+08,2.500000e+08,23.380000,-94.21291,2021-03-13T23:44:36.774Z,4.753810e-01,1.845750e+02,2020-11-04T14:53:53.560Z,None,2022-07-27T19:49:58.766Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,7226.530000,227994088,150,2.646573e+08,64416994.0,7228.420000,...,3.666600e+04,3.666600e+04,90787.000000,-92.04071,2021-05-12T00:29:37.713Z,3.165000e+01,2.273445e+04,2020-07-18T12:26:27.150Z,None,2022-07-27T19:49:24.103Z


### Retrieve historical data & livetime check

In [42]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding SYN, prices data available only for 332 < 365 days
Note: RISE, marketcap data available only for 293 < 365 days
Excluding ALD, prices data available only for 321 < 365 days
Excluding RVST, prices data available only for 304 < 365 days


In [43]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [44]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.900000,3625909763,24,7.943028e+09,300946916.0,7.940000,...,1.000000e+09,1.000000e+09,44.920000,-82.33193,2021-05-03T05:25:04.822Z,1.030000e+00,6.703728e+02,2020-09-17T01:20:38.214Z,None,2022-07-27T19:48:37.576Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,93.530000,1308003999,47,1.498136e+09,234862484.0,93.680000,...,1.600000e+07,1.600000e+07,661.690000,-85.84669,2021-05-18T21:19:59.514Z,2.602000e+01,2.598761e+02,2020-11-05T09:20:11.928Z,None,2022-07-27T19:48:52.550Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.850000,955032606,56,1.853511e+09,103669374.0,1.860000,...,1.000000e+09,1.000000e+09,7.300000,-74.59839,2021-08-20T08:35:20.158Z,4.061500e-01,3.567396e+02,2022-06-18T20:55:12.035Z,None,2022-07-27T19:48:59.813Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,996.630000,898453333,57,1.002389e+09,91597770.0,997.430000,...,9.776310e+05,1.005577e+06,6292.310000,-84.15798,2021-05-03T21:54:29.333Z,1.683600e+02,4.920907e+02,2020-03-16T20:52:36.527Z,None,2022-07-27T19:49:00.310Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008655,659995805,78,8.614101e+08,5768600.0,0.008687,...,9.944413e+10,9.922516e+10,0.120813,-92.81796,2021-06-16T16:03:41.147Z,2.505010e-03,2.463779e+02,2020-11-04T13:18:12.191Z,None,2022-07-27T19:49:03.308Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.683627,385296028,111,1.026095e+09,36641795.0,0.683945,...,1.500000e+09,1.500000e+09,8.650000,-92.09676,2021-10-27T08:24:54.808Z,5.192810e-01,3.167418e+01,2022-06-18T20:10:15.819Z,None,2022-07-27T19:49:52.412Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,53.010000,361079107,116,5.266549e+08,107848420.0,52.970000,...,1.000000e+07,1.000000e+07,910.540000,-94.21283,2021-05-12T02:29:08.794Z,2.641000e+01,9.952380e+01,2022-06-18T20:54:18.261Z,None,2022-07-27T19:49:52.508Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.350000,260565590,137,3.378891e+08,78415560.0,1.350000,...,2.435460e+08,2.500000e+08,23.380000,-94.21291,2021-03-13T23:44:36.774Z,4.753810e-01,1.845750e+02,2020-11-04T14:53:53.560Z,None,2022-07-27T19:49:58.766Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,7226.530000,227994088,150,2.646573e+08,64416994.0,7228.420000,...,3.666600e+04,3.666600e+04,90787.000000,-92.04071,2021-05-12T00:29:37.713Z,3.165000e+01,2.273445e+04,2020-07-18T12:26:27.150Z,None,2022-07-27T19:49:24.103Z


### Marketcap ranking & top10 filtering

In [45]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [46]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,7.900000,3625909763,24,7.943028e+09,300946916.0,7.940000,...,1.000000e+09,1.000000e+09,44.920000,-82.33193,2021-05-03T05:25:04.822Z,1.030000,670.37277,2020-09-17T01:20:38.214Z,None,2022-07-27T19:48:37.576Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,93.530000,1308003999,47,1.498136e+09,234862484.0,93.680000,...,1.600000e+07,1.600000e+07,661.690000,-85.84669,2021-05-18T21:19:59.514Z,26.020000,259.87605,2020-11-05T09:20:11.928Z,None,2022-07-27T19:48:52.550Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.850000,955032606,56,1.853511e+09,103669374.0,1.860000,...,1.000000e+09,1.000000e+09,7.300000,-74.59839,2021-08-20T08:35:20.158Z,0.406150,356.73963,2022-06-18T20:55:12.035Z,None,2022-07-27T19:48:59.813Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,996.630000,898453333,57,1.002389e+09,91597770.0,997.430000,...,9.776310e+05,1.005577e+06,6292.310000,-84.15798,2021-05-03T21:54:29.333Z,168.360000,492.09068,2020-03-16T20:52:36.527Z,None,2022-07-27T19:49:00.310Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008655,659995805,78,8.614101e+08,5768600.0,0.008687,...,9.944413e+10,9.922516e+10,0.120813,-92.81796,2021-06-16T16:03:41.147Z,0.002505,246.37786,2020-11-04T13:18:12.191Z,None,2022-07-27T19:49:03.308Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.683627,385296028,111,1.026095e+09,36641795.0,0.683945,...,1.500000e+09,1.500000e+09,8.650000,-92.09676,2021-10-27T08:24:54.808Z,0.519281,31.67418,2022-06-18T20:10:15.819Z,None,2022-07-27T19:49:52.412Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,53.010000,361079107,116,5.266549e+08,107848420.0,52.970000,...,1.000000e+07,1.000000e+07,910.540000,-94.21283,2021-05-12T02:29:08.794Z,26.410000,99.52380,2022-06-18T20:54:18.261Z,None,2022-07-27T19:49:52.508Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.350000,260565590,137,3.378891e+08,78415560.0,1.350000,...,2.435460e+08,2.500000e+08,23.380000,-94.21291,2021-03-13T23:44:36.774Z,0.475381,184.57496,2020-11-04T14:53:53.560Z,None,2022-07-27T19:49:58.766Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,7226.530000,227994088,150,2.646573e+08,64416994.0,7228.420000,...,3.666600e+04,3.666600e+04,90787.000000,-92.04071,2021-05-12T00:29:37.713Z,31.650000,22734.45246,2020-07-18T12:26:27.150Z,None,2022-07-27T19:49:24.103Z


In [47]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [48]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [49]:
prices_data

,UNI,AAVE,LDO,MKR,AMP,1INCH,COMP,SUSHI,YFI,BAL
date,,,,,,,,,,
2021-02-10,19.513947,490.595917,2.521327,2566.847232,0.012622,5.120852,487.702972,14.324851,35097.258976,38.884601
2021-02-11,20.606185,524.806338,2.163042,2514.308324,0.018005,4.890906,475.521807,14.890322,35967.768454,42.514573
2021-02-12,21.473246,517.005675,2.270908,2528.296118,0.030322,5.638933,501.255182,16.879822,43789.391440,47.315055
2021-02-13,22.702256,529.260260,2.340126,2739.946179,0.034258,5.291805,535.007646,16.574149,47117.151242,52.166919
2021-02-14,21.772030,493.773782,2.533633,2601.196210,0.036486,5.703166,510.631578,15.955116,47063.563316,48.474940
...,...,...,...,...,...,...,...,...,...,...
2022-07-23,6.902377,89.762492,1.521459,950.521623,0.008968,0.663487,55.983886,1.329101,6558.600261,5.471035
2022-07-24,6.994807,89.757338,1.596574,958.131915,0.009093,0.682549,55.780351,1.348547,6517.659004,5.473596
2022-07-25,7.225943,90.897422,1.663252,988.679654,0.008977,0.677460,55.344489,1.353286,6992.431988,5.735606


In [50]:
marketcaps

,UNI,AAVE,LDO,MKR,AMP,1INCH,COMP,SUSHI,YFI,BAL
date,,,,,,,,,,
2021-02-10,5.848772e+09,6.082196e+09,2.231614e+07,2.321050e+09,4.532621e+08,4.945846e+08,2.099922e+09,1.833098e+09,1.106517e+09,4.201539e+08
2021-02-11,6.189234e+09,6.507804e+09,1.945476e+07,2.267010e+09,6.612278e+08,4.725227e+08,2.059282e+09,1.889832e+09,1.139469e+09,4.545828e+08
2021-02-12,6.429248e+09,6.401771e+09,3.127728e+07,2.271078e+09,1.074686e+09,5.429781e+08,2.160530e+09,2.154083e+09,1.379553e+09,5.089683e+08
2021-02-13,6.815509e+09,6.544992e+09,3.240102e+07,2.452029e+09,1.231475e+09,5.112843e+08,2.297470e+09,2.110044e+09,1.488491e+09,5.631770e+08
2021-02-14,6.540139e+09,6.147532e+09,3.411206e+07,2.354080e+09,1.335279e+09,5.523814e+08,2.211330e+09,2.037300e+09,1.493759e+09,5.251610e+08
...,...,...,...,...,...,...,...,...,...,...
2022-07-23,3.146249e+09,1.251268e+09,7.689591e+08,8.566975e+08,6.824492e+08,3.731113e+08,3.836056e+08,2.556540e+08,2.066690e+08,1.916898e+08
2022-07-24,3.191013e+09,1.252076e+09,8.103317e+08,8.628423e+08,6.914614e+08,3.844424e+08,3.822688e+08,2.598291e+08,2.059699e+08,1.916188e+08
2022-07-25,3.306860e+09,1.275891e+09,8.516702e+08,8.962622e+08,6.842056e+08,3.828030e+08,3.806055e+08,2.620482e+08,2.214354e+08,2.016367e+08


## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [51]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [52]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [53]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [54]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [55]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,UNI,AAVE,MKR,LDO,AMP,1INCH,COMP,SUSHI,YFI,BAL
date,,,,,,,,,,
2021-02-10,0.283102,0.294401,0.112348,0.000000,0.021940,0.023940,0.101644,0.088729,0.053560,0.020337
2021-02-11,0.286289,0.300000,0.104863,0.000000,0.030586,0.021857,0.095254,0.087416,0.052707,0.021027
2021-02-12,0.280473,0.279274,0.099075,0.000000,0.046883,0.023687,0.094252,0.093971,0.060182,0.022203
2021-02-13,0.283808,0.272544,0.102106,0.000000,0.051281,0.021291,0.095670,0.087866,0.061983,0.023452
2021-02-14,0.281939,0.265015,0.101482,0.000000,0.057563,0.023813,0.095328,0.087826,0.064395,0.022639
...,...,...,...,...,...,...,...,...,...,...
2022-07-23,0.300000,0.176231,0.120659,0.108302,0.096118,0.052550,0.054028,0.036007,0.029108,0.026998
2022-07-24,0.300000,0.173870,0.119819,0.112527,0.096020,0.053386,0.053084,0.036081,0.028602,0.026609
2022-07-25,0.300000,0.173201,0.121667,0.115614,0.092881,0.051965,0.051667,0.035573,0.030060,0.027372


In [56]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [57]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [58]:
last_weights = adjusted_weights.iloc[-1].dropna()
last_weights

UNI      0.300000
AAVE     0.170569
MKR      0.122066
LDO      0.110336
AMP      0.096048
1INCH    0.053978
COMP     0.050765
SUSHI    0.037096
YFI      0.032707
BAL      0.026436
Name: 2022-07-27 00:00:00, dtype: float64

In [59]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

UNI      76
AAVE     44
MKR      32
LDO      29
AMP      25
1INCH    14
COMP     12
SUSHI     9
YFI       8
BAL       6
Name: 2022-07-27 00:00:00, dtype: int64

## Summary table

In [60]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,3014804316,6.629252,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
AAVE,Aave,1126919650,80.777557,0.170569,44,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
MKR,Maker,806467387,896.947008,0.122066,32,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
LDO,Lido DAO,728973792,1.436833,0.110336,29,0x5a98fcbea516cf06857215779fd812ca3bef1b32
AMP,Amp,634574612,0.008333,0.096048,25,0xff20817765cb7f73d4bde2e66e067e58d11095c2
1INCH,1inch,356625076,0.635008,0.053978,14,0x111111111117dc0aa78b770fa6a738034120c302
COMP,Compound,335392913,48.923667,0.050765,12,0xc00e94cb662c3520282e6f5717214004a7f26888
SUSHI,Sushi,245090389,1.272602,0.037096,9,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2
YFI,yearn.finance,216086669,6843.183709,0.032707,8,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e
